In [7]:
# This is based on python3.8.9
!pip install -r requirements.txt

In [4]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [5]:
class SuperLearner(object):
    def __init__(self, model=None):
        self.models = model
        
    def predict(self, X_test):
        y_pred = pd.DataFrame()
        i = 0
        for model in self.models:
            y_pred[i] = model.predict(X_test)
            i += 1
            
        ret = y_pred.mean(axis=1)
        return ret
       

In [6]:
class Model(object):
    def __init__(self, model_file, scaler_file):
        # Load pre-trained model from file
        self.model = pickle.load(open(model_file, 'rb'))
        # Load a "pre-trained" scaler from file
        self.scaler = pickle.load(open(scaler_file, 'rb'))
        self.sl = SuperLearner(self.model)
        
    def _preprocess(self, features):
        features.replace(['-999', -999], -1, inplace=True)
        features.drop(['Well Identifier'], axis='columns', inplace=True)
        features = MinMaxScaler().fit_transform(features)
        return self.scaler.fit_transform(features)
        
    def predict(self, features):
        X = self._preprocess(features)
        y_pred = self.sl.predict(X)
        return y_pred

In [7]:
mm = Model('slmodel.pkl', 'scaler.pkl')
test_data = pd.read_csv('test_dataset.csv')
y_pred = mm.predict(test_data)


np.savetxt('result.csv', y_pred, header='Rate of Penetration', comments='', fmt='%f')